<a href="https://colab.research.google.com/github/Aswin2808/AI/blob/main/balls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import os

In [41]:
train_dir = '/content/drive/MyDrive/balls/train'
test_dir = '/content/drive/MyDrive/balls/test'
img_path ='/content/drive/MyDrive/cktball.jpg'

In [3]:
# Constants
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 32
EPOCHS = 20

In [4]:
# Data Augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [5]:
# Only rescale for testing data
test_datagen = ImageDataGenerator(rescale=1./255)


In [6]:
# Data Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 1617 images belonging to 3 classes.
Found 406 images belonging to 3 classes.


In [7]:
# Model Architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 classes: volleyball, football, cricket ball
])

In [8]:
# Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Train the Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)

Epoch 1/20
50/50 [==============================] - 80s 2s/step - loss: 0.5042 - accuracy: 0.7950
Epoch 2/20
50/50 [==============================] - 77s 2s/step - loss: 0.4917 - accuracy: 0.8088
Epoch 3/20
50/50 [==============================] - 77s 2s/step - loss: 0.4990 - accuracy: 0.7987
Epoch 4/20
50/50 [==============================] - 76s 2s/step - loss: 0.4789 - accuracy: 0.8013
Epoch 5/20
50/50 [==============================] - 77s 2s/step - loss: 0.4642 - accuracy: 0.8208
Epoch 6/20
50/50 [==============================] - 77s 2s/step - loss: 0.4542 - accuracy: 0.8259
Epoch 7/20
50/50 [==============================] - 76s 2s/step - loss: 0.4652 - accuracy: 0.8252
Epoch 8/20
50/50 [==============================] - 77s 2s/step - loss: 0.4681 - accuracy: 0.8177
Epoch 9/20
50/50 [==============================] - 77s 2s/step - loss: 0.4587 - accuracy: 0.8170
Epoch 10/20
50/50 [==============================] - 76s 1s/step - loss: 0.4526 - accuracy: 0.8215
Epoch 11/20
50/50 [

In [12]:
# Save the Model
model.save('ball_classification_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
# Evaluate the Model
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
print(f'Test Accuracy: {test_acc}')

12/12 [==============================] - 95s 9s/step - loss: 0.4284 - accuracy: 0.8438
Test Accuracy: 0.84375


In [36]:
# Prediction function
def predict_image(image_path):
    if not os.path.exists(image_path):
        return "File not found"

    img = load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize the image

    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    class_labels = ['volleyball', 'football', 'cricket ball']
    return class_labels[predicted_class] if predicted_class in range(len(class_labels)) else "Not found"

# Make a Prediction

In [42]:

predicted_class = predict_image(img_path)
print(f'Predicted Class: {predicted_class}')

1/1 [==============================] - 0s 51ms/step
Predicted Class: volleyball
